In [1]:
# For debug mode
%load_ext autoreload
%autoreload 2

In [1]:
wig_list_paths = [
    "../database/company_list/wig_chemia.csv",
    "../database/company_list/wig_odziez.csv",
    "../database/company_list/wig_spozywczy.csv",
]

### Scraping financial report

In [39]:
import pandas as pd
import scripts as ds

for wig_path in wig_list_paths:
    company_list = pd.read_csv(wig_path)["TICKER"].to_list()
    ds.Scraper(ds.FinancialReport(company_list)).scrape()

### Scraping stock price

In [18]:
import yfinance as yf
import pandas as pd

STOCK_PATH = "../database/stock_price/"

for wig_path in wig_list_paths:
    company_list = pd.read_csv(wig_path)["TICKER"].to_list()
    for ticker in company_list:
        yaho = yf.Ticker("{}.WA".format(ticker))
        history = pd.DataFrame(yaho.history(period="max"))
        history.to_csv("{}{}.csv".format(STOCK_PATH, ticker))  

### Date

In [41]:
import pickle
import pandas as pd

DATE_PATH = "../database/financial_report/date/"

def transform_data_format(x: str):
    sliced = x.split(".")
    return "{}-{}-{}".format(sliced[2], 
                             sliced[1].zfill(2), 
                             sliced[0].zfill(2))  

with open("{}datePublicationDict".format(DATE_PATH), 'rb') as f:
    dict = pickle.load(f)

    
for wig_path in wig_list_paths:
    company_list = pd.read_csv(wig_path)
    for _, rows in company_list.iterrows():
        date = list(dict.fromkeys(dict[rows["NAME"]]["Raport roczny"]))
        date = list(map(transform_data_format, date))
        pd.DataFrame({"Date": date}).to_csv("{}{}.csv".format(DATE_PATH, rows["TICKER"]))